### Importación de librerías

In [ ]:
import cv2
import os
import mediapipe as mp
import numpy as np
import pickle
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

### Definición de variables

In [ ]:
DATASET_PATH = "../../dataset/gestos_cuerpo"
MODEL_FILE = "modelo_gestos_mano.pkl"

### Lógica de entreno

In [ ]:
# Configuración de MediaPipe para IMÁGENES ESTÁTICAS
mp_hands = mp.solutions.hands
# Importante: static_image_mode=True hace que sea más preciso para fotos sueltas
hands = mp_hands.Hands(static_image_mode=True, max_num_hands=1, min_detection_confidence=0.5)

data = []
labels = []

# Mapeo de carpetas a etiquetas numéricas
mapa_etiquetas = {
    "0_Neutro": 0,
    "1_Puno": 1,
    "2_Palma": 2,
    "3_Victoria": 3
}

print("--- INICIANDO PROCESAMIENTO DE FOTOS ---")

# Recorrer cada carpeta del dataset
for nombre_carpeta, etiqueta_num in mapa_etiquetas.items():
    ruta_carpeta = os.path.join(DATASET_PATH, nombre_carpeta)
    
    if not os.path.exists(ruta_carpeta):
        print(f"Advertencia: La carpeta {ruta_carpeta} no existe. Saltando...")
        continue
        
    archivos = os.listdir(ruta_carpeta)
    print(f"Procesando '{nombre_carpeta}': {len(archivos)} imágenes encontradas.")
    
    count_ok = 0
    count_fail = 0
    
    for archivo in archivos:
        if not (archivo.endswith(".jpg") or archivo.endswith(".png")):
            continue
            
        ruta_img = os.path.join(ruta_carpeta, archivo)
        img = cv2.imread(ruta_img)
        
        if img is None:
            continue

        # Convertir a RGB para MediaPipe
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        results = hands.process(img_rgb)
        
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                # Extraer coordenadas (x, y) de los 21 puntos
                landmark_list = []
                for lm in hand_landmarks.landmark:
                    landmark_list.append(lm.x)
                    landmark_list.append(lm.y)
                
                data.append(landmark_list)
                labels.append(etiqueta_num)
                count_ok += 1
        else:
            # Si en alguna foto no sale la mano o está borrosa, la ignoramos
            count_fail += 1

    print(f"  -> {count_ok} procesadas correctamente | {count_fail} fallidas (no se detectó mano)")

# --- ENTRENAMIENTO ---
if len(data) > 0:
    print("\n--- ENTRENANDO MODELO ---")
    data = np.array(data)
    labels = np.array(labels)

    # (Opcional) Separar un 20% para testear qué tan bueno es el modelo
    x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, shuffle=True, stratify=labels)

    model = RandomForestClassifier()
    model.fit(x_train, y_train)

    # Validar precisión
    y_pred = model.predict(x_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Precisión del modelo: {accuracy * 100:.2f}%")

    # Guardar modelo final
    model.fit(data, labels) # Re-entrenamos con todos los datos para el archivo final
    
    with open(MODEL_FILE, 'wb') as f:
        pickle.dump(model, f)
    
    print(f"¡ÉXITO! Modelo guardado como '{MODEL_FILE}'.")
    print("Ahora puedes ejecutar 'simon_game.py'")

else:
    print("ERROR: No se pudieron extraer datos. Revisa tus fotos o la iluminación.")